# 지역별 연봉 분석
- 법정동 데이터 : https://www.data.go.kr/data/15063424/fileData.do
- 국민연금 가입사업자 내역 데이터 :https://www.data.go.kr/data/15083277/fileData.do

In [1]:
import pandas as pd
import numpy as np
import cufflinks as cf  # 시각화 관련 library
# cufflinks 관련 옵션
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [2]:
salary = pd.read_csv('./large_data/국민연금공단_국민연금 가입 사업장 내역_20220331.csv',
                   encoding='cp949', low_memory=False)

In [3]:
salary.columns = [x.strip() for x in salary.columns]

In [4]:
salary.columns

Index(['자료생성년월', '사업장명', '사업자등록번호', '사업장가입상태코드 1 등록 2 탈퇴', '우편번호', '사업장지번상세주소',
       '사업장도로명상세주소', '고객법정동주소코드', '고객행정동주소코드', '법정동주소광역시도코드', '법정동주소광역시시군구코드',
       '법정동주소광역시시군구읍면동코드', '사업장형태구분코드 1 법인 2 개인', '사업장업종코드', '사업장업종코드명',
       '적용일자', '재등록일자', '탈퇴일자', '가입자수', '당월고지금액', '신규취득자수', '상실가입자수'],
      dtype='object')

## 평균 연봉 - 가산vs분당

In [5]:
pay = salary['당월고지금액']/salary['가입자수']
pay = pay * 12 * 100 / 9 # 국민연금 납부액을 통한 연봉추정액
salary['pay'] = pay

In [6]:
gasan = salary[salary['사업장도로명상세주소'].str.contains('서울특별시 금천구 가산')]
pangyo = salary[salary['사업장도로명상세주소'].str.contains('경기도 성남시 분당구 판교')]

In [7]:
print(gasan['pay'].mean())

35476823.07607453


In [8]:
print(pangyo['pay'].mean())

39328922.509835914


In [9]:
gasan['가입자수'].mean()

17.627337488869102

In [13]:
pangyo['가입자수'].mean()

51.67623833452979

In [14]:
def get_mean_salary(df):
    mean_salary = (df['pay']*df['가입자수']).sum() / df['가입자수'].sum()
    return mean_salary

In [15]:
print(get_mean_salary(gasan))

37541855.80898015


In [16]:
print(get_mean_salary(pangyo))

46036217.2489639


## 지역별 평균연봉

In [19]:
address_code = pd.read_csv('data/국토교통부_전국 법정동_20211217.csv', encoding='cp949')
address_code = address_code.fillna('')
address_code['주소'] = address_code[['시도명', '시군구명', '읍면동명', '리명']].agg(' '.join, axis=1)
address_code['주소길이'] = address_code['주소'].str.len()
address_code = address_code.sort_values('주소길이')
address_code['code'] = (address_code['법정동코드'] // 100000).astype(str)
address_code = address_code.drop_duplicates('code', keep='first')[['code', '주소']].reset_index(drop=True)

In [ ]:
salary['code'] = salary['고객법정동주소코드'].astype(str).str[:5]
salary['pay_sum'] = salary['pay'] * salary['가입자수']
salary_region = salary.groupby('code').mean()
salary_region['pay_mean'] = salary_region['pay_sum'] / salary_region['가입자수']

In [31]:
temp = salary.groupby('code').sum()
valid_codes = temp[temp['가입자수'] > 100].reset_index()['code']  # 근로자수가 100명 이상인 지역을 대상으로

In [45]:
salary_region = salary_region[salary_region.index.isin(valid_codes)]

In [46]:
salary_top10 = salary_region.sort_values('pay_mean').tail(10)
salary_top10 = salary_top10.reset_index()[::-1]
salary_top10 = salary_top10.merge(address_code, on='code')

In [48]:
salary_top10.iplot(kind='bar', x='주소', y='pay_mean')

In [49]:
salary_top10[['code', '주소']]

,code,주소
0,41117,경기도 수원시영통구
1,31170,울산광역시 동구
2,41500,경기도 이천시
3,46170,전라남도 나주시
4,47111,경상북도 포항시남구
5,28140,인천광역시 동구
6,44270,충청남도 당진시
7,48123,경상남도 창원시성산구
8,44210,충청남도 서산시
9,41463,경기도 용인시기흥구


In [53]:
salary[salary['code'] == '46170'].sort_values('가입자수', ascending=False)[['사업장명', 'pay', '가입자수']]  # 나주시

,사업장명,pay,가입자수
3459,한국전력공사,5.792509e+07,23136
146115,한국농어촌공사,4.850979e+07,6593
3379,한전KPS(주),5.415979e+07,6365
387120,한전엠씨에스 주식회사,5.498506e+07,3946
57040,한전케이디엔(주),4.859010e+07,3012
...,...,...,...
508448,유진타워건설주식회사/일용/이엔테크(주) 나주공장 증축공사,NaN,0
508227,유진타워건설주식회사/일용/(주)에이스테크 나주공장 증축공사,NaN,0
506270,(주)금강종합건설/일용/나주학산리504-21번지공장신축공사,NaN,0
502020,주식회사서해건설/일용/남평지석천친수시설정비공사,NaN,0


In [52]:
salary[salary['code'] == '44270'].sort_values('가입자수', ascending=False)[['사업장명', 'pay', '가입자수']]  # 당진시

,사업장명,pay,가입자수
46921,현대제철주식회사당진공장,6.179895e+07,6299
492136,현대아이티씨주식회사,5.291539e+07,3499
153906,한국내화주식회사,5.649345e+07,548
164228,한전산업개발(주) 당진사업처,4.933191e+07,458
145437,대한전선(주)당진공장,5.477779e+07,426
...,...,...,...
510056,(주)다원엔지니어링-(일용)배수종말설비 노후설비 정상화 공사 기계배관설치공사,NaN,0
509287,남강종합건설（주）-(일용)석문중학교 외벽수선 공사,NaN,0
509206,（주）승진기계-(일용)21년_공사_화성1 2기BTX포집탑패킹교체,NaN,0
508975,(주)경선산업개발-(일용)신평먼 부수리 상오리 초대리 배수로 정비공사,NaN,0


In [77]:
salary_bot10 = salary_region.sort_values('pay_mean').head(10)
salary_bot10 = salary_bot10.reset_index()
salary_bot10 = salary_bot10.merge(address_code, on='code')

In [79]:
salary_bot10[['code', '주소']]

,code,주소
0,41197,경기도 부천시소사구
1,43710,충청북도 청원군
2,42800,강원도 양구군
3,28170,인천광역시 남구
4,47760,경상북도 영양군
5,41195,경기도 부천시원미구
6,46110,전라남도 목포시
7,27200,대구광역시 남구
8,47750,경상북도 청송군
9,45740,전라북도 장수군


In [80]:
salary[salary['code'] == '41197'].sort_values('가입자수', ascending=False)[['사업장명', 'pay', '가입자수']]

,사업장명,pay,가입자수
209424,주식회사 햇빛나눔,1.555400e+07,143
34636,부천일신중학교,3.003496e+07,27
32523,부천일신초등학교,3.027511e+07,24
66999,부천양지초등학교 행정실,2.704987e+07,20
159440,창영초등학교,2.838067e+07,20
73674,소안초등학교,2.996667e+07,18
159444,범박초등학교,2.858243e+07,17
90606,(주)아이에스종합관리,2.574086e+07,17
158866,복사초등학교,3.006861e+07,11
259173,에스티대흥(주）,3.216356e+07,6
